In [1]:
import pandas as pd
import pathlib
import numpy as np

## Load Data

In [2]:
raw_data_dir = pathlib.Path("../../data/raw/soroush_2")

trans_res_path = raw_data_dir.joinpath('2023.09.06 BSA_OVA_quant_super_akin.csv')
trans_res_df_raw = pd.read_csv(trans_res_path)
trans_res_df_raw.columns = trans_res_df_raw.columns.map(str.lower)
trans_res_df_raw.head()

,peptide,protein,replicate,precursor mz,precursor charge,product mz,product charge,fragment ion,retention time,area,background
0,GGLEPINFQTAADQAR,sp|P01012|OVAL_CHICK,Soroush_SDC_W1730_Sup_D0_column12_2uL_20,844.42355,2,844.42355,2,precursor,47.00,7.575379e+09,133355208
1,GGLEPINFQTAADQAR,sp|P01012|OVAL_CHICK,Soroush_SDC_W1730_Sup_D0_column12_2uL_80,844.42355,2,844.42355,2,precursor,45.96,9.467220e+09,98842
2,GGLEPINFQTAADQAR,sp|P01012|OVAL_CHICK,Soroush_SDC_W1730_Sup_D1_column12_2uL_141,844.42355,2,844.42355,2,precursor,46.89,5.926985e+07,0
3,GGLEPINFQTAADQAR,sp|P01012|OVAL_CHICK,Soroush_SDC_W1730_Sup_D1_column12_2uL_159,844.42355,2,844.42355,2,precursor,45.72,3.679658e+09,20224484
4,GGLEPINFQTAADQAR,sp|P01012|OVAL_CHICK,Soroush_SDC_W1730_Sup_D2_column12_2uL_32,844.42355,2,844.42355,2,precursor,46.89,3.454373e+09,30604174


In [3]:

trans_res_df_raw.peptide.unique()

array(['GGLEPINFQTAADQAR', 'AEFVEVTK', 'YICDNQDTISSK'], dtype=object)

## Compute Area ratio

In [4]:
# def grp_area_ratio(grouped_df:pd.DataFrame):
    
#     """compute the ratio of areas with higer precursor mz value as numerator."""

#     df = grouped_df[['precursor mz', 'area']].copy()
#     df = df.sort_values(by='precursor mz', ascending=True)
#     area_ratio = df['area'].iloc[1] / df['area'].iloc[0]

#     return area_ratio

def grp_area_ratio_n_area_cols(grouped_df:pd.DataFrame):
    
    """compute the ratio of areas with higer precursor mz value as numerator."""

    df = grouped_df[['precursor mz', 'area']].copy()
    df = df.sort_values(by='precursor mz', ascending=True)
    area_ratio = df['area'].iloc[1] / df['area'].iloc[0]
    area_min, area_max  =  df['area'].iloc[0], df['area'].iloc[1]

    col_vals = {'area_ratio': area_ratio, 'area_min':area_min, 'area_max':area_max}

    return pd.Series(col_vals)

cols_to_group_by = ['peptide', 'protein', 'replicate', 'product charge', 'fragment ion']
# df_area_ratio = trans_res_df_raw.groupby(cols_to_group_by).apply(grp_area_ratio).reset_index(name='area_ratio')
df_area_ratio = trans_res_df_raw.groupby(cols_to_group_by).apply(grp_area_ratio_n_area_cols).reset_index()


# df_area_ratio = df_area_ratio[df_area_ratio['fragment ion'] == 'precursor']  # select only precursor
print(df_area_ratio.shape)
df_area_ratio.head()

c:\Users\Aroge\anaconda3\envs\genomic\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: divide by zero encountered in double_scalars
c:\Users\Aroge\anaconda3\envs\genomic\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in double_scalars


(1080, 8)


,peptide,protein,replicate,product charge,fragment ion,area_ratio,area_min,area_max
0,AEFVEVTK,sp|P02769|ALBU_BOVIN,Soroush_SDC_W1730_Sup_D0_column12_2uL_20,1,b2,0.000344,3.157382e+09,1085662.0
1,AEFVEVTK,sp|P02769|ALBU_BOVIN,Soroush_SDC_W1730_Sup_D0_column12_2uL_20,1,y2,0.000128,4.009150e+08,51280.0
2,AEFVEVTK,sp|P02769|ALBU_BOVIN,Soroush_SDC_W1730_Sup_D0_column12_2uL_20,1,y4,0.000017,6.640748e+08,11599.0
3,AEFVEVTK,sp|P02769|ALBU_BOVIN,Soroush_SDC_W1730_Sup_D0_column12_2uL_20,1,y5,0.000003,8.254646e+08,2376.0
4,AEFVEVTK,sp|P02769|ALBU_BOVIN,Soroush_SDC_W1730_Sup_D0_column12_2uL_20,1,y6,0.000171,2.778528e+09,475204.0


In [5]:
np.isinf(df_area_ratio.area_ratio).sum()

15

## Get peptide Dilution Conc.

In [6]:
# df_peptide_dilution_conc = pd.read_csv(raw_data_dir.joinpath('Soroush_2023_09_06_peptide_dilution_conc.csv'))
# # df_peptide_dilution_conc.columns


# df_peptide_dilution_conc.head()
# raw_dilution_col_names = df_peptide_dilution_conc.columns[1:]


# df_peptide_dilution_conc = df_peptide_dilution_conc.melt(id_vars='Peptides', value_vars=raw_dilution_col_names, 
#                                                         var_name='dilution', value_name='heavy_conc')
# df_peptide_dilution_conc['dilution'] = df_peptide_dilution_conc['dilution'].apply(lambda x: x[:2])
# df_peptide_dilution_conc['peptide_dilution_name'] = df_peptide_dilution_conc['Peptides'] + '_' + \
# df_peptide_dilution_conc['dilution'] 
# df_peptide_dilution_conc.head()

In [7]:



def get_peptide_dilution_comb(row):
    """function to create a new column from peptide and dilution name."""
    peptide = row['peptide']
    replicate  = row['replicate']
    dilution = get_dilution_from_replicate(rep_string=replicate)

    return peptide + '_' + dilution

df_peptide_dilution_conc = pd.read_csv(raw_data_dir.joinpath('Soroush_2023_09_06_peptide_dilution_conc.csv'))
raw_dilution_col_names = df_peptide_dilution_conc.columns[1:]

df_peptide_dilution_conc = df_peptide_dilution_conc.melt(id_vars='Peptides', value_vars=raw_dilution_col_names, 
                                                        var_name='dilution', value_name='heavy_conc')
df_peptide_dilution_conc['dilution'] = df_peptide_dilution_conc['dilution'].apply(lambda x: x[:2])
peptide_dilution_name_col = 'peptide_dilution_name'
df_peptide_dilution_conc[peptide_dilution_name_col] = df_peptide_dilution_conc['Peptides'] + '_' + \
                                                    df_peptide_dilution_conc['dilution'] 
df_peptide_dilution_conc.head()


,Peptides,dilution,heavy_conc,peptide_dilution_name
0,GGLEPINFQTAADQAR,D1,0.736000,GGLEPINFQTAADQAR_D1
1,AEFVEVTK,D1,0.403667,AEFVEVTK_D1
2,YICDNQDTISSK,D1,1.257333,YICDNQDTISSK_D1
3,GGLEPINFQTAADQAR,D2,0.184000,GGLEPINFQTAADQAR_D2
4,AEFVEVTK,D2,0.100917,AEFVEVTK_D2


### Merge data

In [8]:
def get_dilution_from_replicate(rep_string:str):
    """Get the dilution symbol from the replicate name."""

    str_to_check = rep_string.split('_')[4]
    if str_to_check.startswith('col'):
        dilution = 'D0'
    else:
        dilution = str_to_check

    return dilution

df_area_ratio[peptide_dilution_name_col] = df_area_ratio.apply(get_peptide_dilution_comb, axis=1)



df_area_ratio_conc = pd.merge(df_area_ratio, right=df_peptide_dilution_conc, how='left', 
                              on=peptide_dilution_name_col, suffixes=('', '_y'))
df_area_ratio_conc.drop(df_area_ratio_conc.filter(regex='_y$').columns, axis=1, inplace=True)
df_area_ratio_conc = df_area_ratio_conc.dropna(axis=0) # drop nan
# drop inf
df_area_ratio_conc.replace([np.inf, -np.inf], np.nan, inplace=True)
# df_area_ratio_conc.dropna(subset=["area_ratio"], how="all", inplace=True)

# def get_rep_plot_cat(rep_val:str):
#     """Get the plot category (C18_SDS, C18_SDC, etc..)."""

#     select_elements = rep_val.split('_')[1:3]

#     cat = '_'.join(select_elements)
#     return cat

# TODO: - reverse x and y in plots: ratio:y, conc: x
#- TODO: further break replicate into 2 groups as shown in the excel, the greens are one group and the white is another.
def get_rep_plot_cat(row):
    """Get the plot category (C18_SDS, C18_SDC, etc..)."""

    replicate = row['replicate']
    peptide = row['peptide']
    fragment_ion = row['fragment ion']

    rep_elements = replicate.split('_')[0:3]

    cat_name = [peptide] + [fragment_ion] + rep_elements
    cat_name = '_'.join(cat_name)
    return cat_name

def get_order(grouped_df):

    order_vals = []
    for rep_value in grouped_df['replicate'].values:
        rep_partial = rep_value.split('_')[:-1]
        rep_last_val = int(rep_value.split('_')[-1])

        for rep_value_2 in grouped_df['replicate'].values:
            rep_partial_2 = rep_value_2.split('_')[:-1]
            rep_last_val_2 = int(rep_value_2.split('_')[-1])

            found_match = False
            if rep_value != rep_value_2 and rep_partial == rep_partial_2:
                found_match = True
                if rep_last_val < rep_last_val_2:
                    order_val = 'low'
                elif rep_last_val > rep_last_val_2:
                    order_val = 'high'
                break
                
        if found_match:
            order_vals.append(order_val)
        else:
            order_vals.append('no match')

        
    
    grouped_df['order'] = order_vals
    return grouped_df
# create plot categories
df_area_ratio_conc['plot_cat'] = df_area_ratio_conc.apply(get_rep_plot_cat, axis=1)
order = df_area_ratio_conc.groupby('plot_cat').apply(get_order)['order']
df_area_ratio_conc['plot_cat'] = df_area_ratio_conc['plot_cat'] + '_' + order
df_area_ratio_conc.head()
# print(df_area_ratio_conc.isna().sum())
# df_area_ratio_conc = df_area_ratio_conc[df_area_ratio_conc['area_ratio'].notna()]
# print(df_area_ratio_conc.isna().sum())
df_area_ratio_conc.head()


,peptide,protein,replicate,product charge,fragment ion,area_ratio,area_min,area_max,peptide_dilution_name,Peptides,dilution,heavy_conc,plot_cat
0,AEFVEVTK,sp|P02769|ALBU_BOVIN,Soroush_SDC_W1730_Sup_D0_column12_2uL_20,1,b2,0.000344,3.157382e+09,1085662.0,AEFVEVTK_D0,AEFVEVTK,D0,0.0,AEFVEVTK_b2_Soroush_SDC_W1730_low
1,AEFVEVTK,sp|P02769|ALBU_BOVIN,Soroush_SDC_W1730_Sup_D0_column12_2uL_20,1,y2,0.000128,4.009150e+08,51280.0,AEFVEVTK_D0,AEFVEVTK,D0,0.0,AEFVEVTK_y2_Soroush_SDC_W1730_low
2,AEFVEVTK,sp|P02769|ALBU_BOVIN,Soroush_SDC_W1730_Sup_D0_column12_2uL_20,1,y4,0.000017,6.640748e+08,11599.0,AEFVEVTK_D0,AEFVEVTK,D0,0.0,AEFVEVTK_y4_Soroush_SDC_W1730_low
3,AEFVEVTK,sp|P02769|ALBU_BOVIN,Soroush_SDC_W1730_Sup_D0_column12_2uL_20,1,y5,0.000003,8.254646e+08,2376.0,AEFVEVTK_D0,AEFVEVTK,D0,0.0,AEFVEVTK_y5_Soroush_SDC_W1730_low
4,AEFVEVTK,sp|P02769|ALBU_BOVIN,Soroush_SDC_W1730_Sup_D0_column12_2uL_20,1,y6,0.000171,2.778528e+09,475204.0,AEFVEVTK_D0,AEFVEVTK,D0,0.0,AEFVEVTK_y6_Soroush_SDC_W1730_low


In [9]:
df_area_ratio_conc[df_area_ratio_conc.plot_cat.apply(lambda x: x.split('_')[-1] == 'no match')]

,peptide,protein,replicate,product charge,fragment ion,area_ratio,area_min,area_max,peptide_dilution_name,Peptides,dilution,heavy_conc,plot_cat
847,YICDNQDTISSK,sp|P02769|ALBU_BOVIN,Soroush_SDC_W42_Sup_D2_column12_2uL_33,1,b3,47.961354,8337061.0,399856736.0,YICDNQDTISSK_D2,YICDNQDTISSK,D2,0.314333,YICDNQDTISSK_b3_Soroush_SDC_W42_no match
1054,YICDNQDTISSK,sp|P02769|ALBU_BOVIN,Soroush_SDC_WW_Sup_D3_column12_2uL_95,1,b3,7.019977,7057353.0,49542456.0,YICDNQDTISSK_D3,YICDNQDTISSK,D3,0.078583,YICDNQDTISSK_b3_Soroush_SDC_WW_no match


In [11]:
df_area_ratio_conc['plot_cat'][:20]

0                   AEFVEVTK_b2_Soroush_SDC_W1730_low
1                   AEFVEVTK_y2_Soroush_SDC_W1730_low
2                   AEFVEVTK_y4_Soroush_SDC_W1730_low
3                   AEFVEVTK_y5_Soroush_SDC_W1730_low
4                   AEFVEVTK_y6_Soroush_SDC_W1730_low
5            AEFVEVTK_precursor_Soroush_SDC_W1730_low
6      AEFVEVTK_precursor [M+1]_Soroush_SDC_W1730_low
7      AEFVEVTK_precursor [M+2]_Soroush_SDC_W1730_low
8                   AEFVEVTK_y6_Soroush_SDC_W1730_low
9                  AEFVEVTK_b2_Soroush_SDC_W1730_high
10                 AEFVEVTK_y2_Soroush_SDC_W1730_high
11                 AEFVEVTK_y4_Soroush_SDC_W1730_high
12                 AEFVEVTK_y5_Soroush_SDC_W1730_high
13                 AEFVEVTK_y6_Soroush_SDC_W1730_high
14          AEFVEVTK_precursor_Soroush_SDC_W1730_high
15    AEFVEVTK_precursor [M+1]_Soroush_SDC_W1730_high
16    AEFVEVTK_precursor [M+2]_Soroush_SDC_W1730_high
17                 AEFVEVTK_y6_Soroush_SDC_W1730_high
18                  AEFVEVTK

In [13]:
df_area_ratio_conc['plot_cat'].nunique()

194

## Get Linear Fit parameters

In [15]:
from sklearn import linear_model, metrics
import matplotlib.pyplot as plt

def get_plot(data:pd.DataFrame, plot_cat:str, ax):

    cat_data = data[data['plot_cat'] == plot_cat]
    # print(cat_data)

    # fig, ax = plt.subplots()
    cat_data.dropna(axis=0, inplace=True)

    y = cat_data['area_ratio'].values
    x = cat_data['heavy_conc'].values

    ax.scatter(x, y)

    x = x[:, np.newaxis]
    y = y[:, np.newaxis]

    model = linear_model.LinearRegression()

    model.fit(x, y)

    y_fit = model.predict(x)

    ax.plot(x.flatten(), y_fit.flatten())
    ax.set_title(plot_cat)
    
    return ax

def get_linear_fit(data:pd.DataFrame, plot_cat:str):

    cat_data = data[data['plot_cat'] == plot_cat]
    # print(cat_data)

    # fig, ax = plt.subplots()
    cat_data.dropna(axis=0, inplace=True)

    y = cat_data['area_ratio'].values
    x = cat_data['heavy_conc'].values

    # ax.scatter(x, y)

    x = x[:, np.newaxis]
    y = y[:, np.newaxis]

    model = linear_model.LinearRegression()

    model.fit(x, y)

    y_fit = model.predict(x)

    # ax.plot(x.flatten(), y_fit.flatten())

    r2 = metrics.r2_score(y, y_fit)
    intercept = model.intercept_.squeeze()
    grad = model.coef_[0].squeeze()


    # print(f"R^2: {r2}")
    # print(f"intercept: {intercept}")
    # print(f"gradient:{grad}")

    # plt.show()
    
    return (r2, intercept, grad)

plot_cats = df_area_ratio_conc['plot_cat'].unique()

# plot_df = df_area_ratio_conc[df_area_ratio_conc.peptide=='GGLEPINFQTAADQAR'] # get only one peptide
# plot_df = plot_df[[peptide_dilution_name_col, 'area_ratio', 'heavy_conc', 'plot_cat']]
plot_df = df_area_ratio_conc


cats = []
r2s = []
intercepts = []
grads = []

for cat in plot_cats:
    # print(cat)
    r2, intercept, grad = get_linear_fit(plot_df, cat)
    cats.append(cat)
    r2s.append(r2)
    intercepts.append(intercept)
    grads.append(grad)

# n_plot_cats = len(plot_cats)
# fig_ax_dims = (4,4)
# n_figs = np.ceil(n_plot_cats / np.product(fig_ax_dims))

# axs = []
# for fig_i in n_figs:
#     fig, axs = plt.subplots(nrows=fig_ax_dims[0], ncols=fig_ax_dims[1])
#     fig_axs = axs.flatten()
#     for ax in fig_axs:
#         axs.append(ax)

result_data_folder = pathlib.Path("../../reports/ms_work/plots")
for cat in plot_cats:
    fig, ax = plt.subplots()
    ax = get_plot(plot_df, cat, ax)
    fig = plt.gcf()
    fig.savefig(fname=result_data_folder.joinpath(cat))




df_plot_cat_fit_params = pd.DataFrame({'plot_cat':cats,
              'R2':r2s,
              'intercept':intercepts,
              'gradient':grads})




c:\Users\Aroge\anaconda3\envs\genomic\lib\site-packages\ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\Users\Aroge\anaconda3\envs\genomic\lib\site-packages\ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\Users\Aroge\anaconda3\envs\genomic\lib\site-packages\ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\Users\Aroge\anaconda3\envs\genomic\lib\site-packages\ipyke

In [40]:
# merge to assign group properties to individual rows
df_area_ratio_conc_n_fit_params = pd.merge(df_area_ratio_conc,
                                           right=df_plot_cat_fit_params,
                                           on='plot_cat',
                                           how='left')

df_area_ratio_conc_n_fit_params
# save to csv file
# df_area_ratio_conc_n_fit_params.to_csv('area_ratio_conc_n_fit_params.csv', index=False)
    

,peptide,protein,replicate,product charge,fragment ion,area_ratio,area_min,area_max,peptide_dilution_name,Peptides,dilution,heavy_conc,plot_cat,R2,intercept,gradient
0,AEFVEVTK,sp|P02769|ALBU_BOVIN,Soroush_SDC_W1730_Sup_D0_column12_2uL_20,1,b2,0.000344,3.157382e+09,1085662.0,AEFVEVTK_D0,AEFVEVTK,D0,0.000000,AEFVEVTK_b2_Soroush_SDC_W1730_low,0.989631,-0.13147736467139093,11.51259813335332
1,AEFVEVTK,sp|P02769|ALBU_BOVIN,Soroush_SDC_W1730_Sup_D0_column12_2uL_20,1,y2,0.000128,4.009150e+08,51280.0,AEFVEVTK_D0,AEFVEVTK,D0,0.000000,AEFVEVTK_y2_Soroush_SDC_W1730_low,0.988234,-0.1502222186570894,12.334220374266625
2,AEFVEVTK,sp|P02769|ALBU_BOVIN,Soroush_SDC_W1730_Sup_D0_column12_2uL_20,1,y4,0.000017,6.640748e+08,11599.0,AEFVEVTK_D0,AEFVEVTK,D0,0.000000,AEFVEVTK_y4_Soroush_SDC_W1730_low,0.988140,-0.14864927352590418,12.30423952857855
3,AEFVEVTK,sp|P02769|ALBU_BOVIN,Soroush_SDC_W1730_Sup_D0_column12_2uL_20,1,y5,0.000003,8.254646e+08,2376.0,AEFVEVTK_D0,AEFVEVTK,D0,0.000000,AEFVEVTK_y5_Soroush_SDC_W1730_low,0.988700,-0.1387292636187265,11.684069531545752
4,AEFVEVTK,sp|P02769|ALBU_BOVIN,Soroush_SDC_W1730_Sup_D0_column12_2uL_20,1,y6,0.000171,2.778528e+09,475204.0,AEFVEVTK_D0,AEFVEVTK,D0,0.000000,AEFVEVTK_y6_Soroush_SDC_W1730_low,0.988829,-0.14055022618436963,12.197474762376542
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1073,YICDNQDTISSK,sp|P02769|ALBU_BOVIN,Soroush_SDC_WW_Sup_D4_column12_2uL_89,1,y9,0.708577,2.510951e+07,17792034.0,YICDNQDTISSK_D4,YICDNQDTISSK,D4,0.007858,YICDNQDTISSK_y9_Soroush_SDC_WW_high,0.998827,-3.8072873413344297,229.38052894238007
1074,YICDNQDTISSK,sp|P02769|ALBU_BOVIN,Soroush_SDC_WW_Sup_D4_column12_2uL_89,2,precursor,0.729514,6.567330e+08,479095968.0,YICDNQDTISSK_D4,YICDNQDTISSK,D4,0.007858,YICDNQDTISSK_precursor_Soroush_SDC_WW_high,0.954948,-16.824865355130406,240.57479242298194
1075,YICDNQDTISSK,sp|P02769|ALBU_BOVIN,Soroush_SDC_WW_Sup_D4_column12_2uL_89,2,precursor [M+1],0.689912,4.416241e+08,304681568.0,YICDNQDTISSK_D4,YICDNQDTISSK,D4,0.007858,YICDNQDTISSK_precursor [M+1]_Soroush_SDC_WW_high,0.974994,-14.58724185388455,257.2554796725508
1076,YICDNQDTISSK,sp|P02769|ALBU_BOVIN,Soroush_SDC_WW_Sup_D4_column12_2uL_89,2,precursor [M+2],0.637243,1.895787e+08,120807784.0,YICDNQDTISSK_D4,YICDNQDTISSK,D4,0.007858,YICDNQDTISSK_precursor [M+2]_Soroush_SDC_WW_high,0.956988,-18.248494717278717,261.75467289705176


In [ ]:
q90 = [0.941, 0.765, 0.642, 0.56, 0.507, 0.468, 0.437,
       0.412, 0.392, 0.376, 0.361, 0.349, 0.338, 0.329,
       0.32, 0.313, 0.306, 0.3, 0.295, 0.29, 0.285, 0.281,
       0.277, 0.273, 0.269, 0.266, 0.263, 0.26
      ]

q95 = [0.97, 0.829, 0.71, 0.625, 0.568, 0.526, 0.493, 0.466,
       0.444, 0.426, 0.41, 0.396, 0.384, 0.374, 0.365, 0.356,
       0.349, 0.342, 0.337, 0.331, 0.326, 0.321, 0.317, 0.312,
       0.308, 0.305, 0.301, 0.29
      ]

q99 = [0.994, 0.926, 0.821, 0.74, 0.68, 0.634, 0.598, 0.568,
       0.542, 0.522, 0.503, 0.488, 0.475, 0.463, 0.452, 0.442,
       0.433, 0.425, 0.418, 0.411, 0.404, 0.399, 0.393, 0.388,
       0.384, 0.38, 0.376, 0.372
       ]

Q90 = {n:q for n,q in zip(range(3,len(q90)+1), q90)}
Q95 = {n:q for n,q in zip(range(3,len(q95)+1), q95)}
Q99 = {n:q for n,q in zip(range(3,len(q99)+1), q99)}

def dixon_test(data, left=True, right=True, q_dict=Q95):
    """
    Keyword arguments:
        data = A ordered or unordered list of data points (int or float).
        left = Q-test of minimum value in the ordered list if True.
        right = Q-test of maximum value in the ordered list if True.
        q_dict = A dictionary of Q-values for a given confidence level,
            where the dict. keys are sample sizes N, and the associated values
            are the corresponding critical Q values. E.g.,
            {3: 0.97, 4: 0.829, 5: 0.71, 6: 0.625, ...}

    Returns a list of 2 values for the outliers, or None.
    E.g.,
       for [1,1,1] -> [None, None]
       for [5,1,1] -> [None, 5]
       for [5,1,5] -> [1, None]

    """
    assert(left or right), 'At least one of the variables, `left` or `right`, must be True.'
    assert(len(data) >= 3), 'At least 3 data points are required'
    assert(len(data) <= max(q_dict.keys())), 'Sample size too large'

    sdata = sorted(data)
    Q_mindiff, Q_maxdiff = (0,0), (0,0)

    if left:
        Q_min = (sdata[1] - sdata[0])
        try:
            Q_min /= (sdata[-1] - sdata[0])
        except ZeroDivisionError:
            pass
        Q_mindiff = (Q_min - q_dict[len(data)], sdata[0])

    if right:
        Q_max = abs((sdata[-2] - sdata[-1]))
        try:
            Q_max /= abs((sdata[0] - sdata[-1]))
        except ZeroDivisionError:
            pass
        Q_maxdiff = (Q_max - q_dict[len(data)], sdata[-1])

    if not Q_mindiff[0] > 0 and not Q_maxdiff[0] > 0:
        outliers = [None, None]

    elif Q_mindiff[0] == Q_maxdiff[0]:
        outliers = [Q_mindiff[1], Q_maxdiff[1]]

    elif Q_mindiff[0] > Q_maxdiff[0]:
        outliers = [Q_mindiff[1], None]

    else:
        outliers = [None, Q_maxdiff[1]]

    return outliers


dixon_test([0.142, 0.153, 0.135, 0.002, 0.175], left=False)

# Fit metric

In [41]:
def get_fit_param_cat(row):
    """Get the fit param category (C18_SDS, C18_SDC, etc..)."""

    replicate = row['replicate']
    peptide = row['peptide']

    rep_elements = replicate.split('_')[0:3]

    cat_name = [peptide]  + rep_elements
    cat_name = '_'.join(cat_name)
    return cat_name

def qtest(data, right=True):

    sorted_data = sorted(data)

    if right:
        gap = sorted_data[-1] - sorted_data[-2]

    else:
        gap = sorted_data[1] - sorted_data[0]

    try:
        range = sorted_data[-1] - sorted_data[0]
        q_val = gap / range
    except ZeroDivisionError:
        q_val = gap

    return q_val

def get_grp_fit_agg(grouped_df:pd.DataFrame):
    """Get the aggregate values for each group."""

    df = grouped_df[['intercept', 'gradient', 'R2']].copy()
    gradient = df['gradient'].values
    intercept = df['intercept'].values
    r2 = df['R2'].values

    mean_grad = gradient.mean()
    stdv_grad = gradient.std()
    cov_grad = stdv_grad / mean_grad
    qtest_grad_right = qtest(gradient, right=True)

    mean_intercept = intercept.mean()
    stdv_intercept = intercept.std()
    cov_intercept = stdv_intercept / mean_intercept
    qtest_intercept_right = qtest(intercept, right=True)

    mean_r2 = r2.mean()
    stdv_r2 = r2.std()
    cov_r2 = stdv_r2 / mean_r2
    qtest_r2_right = qtest(r2, right=True)

    cols_dict = {'mean_grad':mean_grad, 'stdv_grad':stdv_grad,
                 'cov_grad':cov_grad, 'qtest_grad':qtest_grad_right, 
                 'mean_intercept':mean_intercept, 'stdv_intercept':stdv_intercept,
                 'cov_intercept':cov_intercept, 'qtest_intercept':qtest_intercept_right,
                 'mean_r2':mean_r2, 'stdv_r2':stdv_r2,
                 'cov_r2':cov_r2, 'qtest_r2':qtest_r2_right
                 }

    return pd.Series(cols_dict)


# create plot categories
temp = df_area_ratio_conc_n_fit_params.copy()
temp['fit_param_grp'] = temp.apply(get_fit_param_cat, axis=1)

cols_to_group_by = ['fit_param_grp']
df_fit_param_agg = temp.groupby(cols_to_group_by).apply(get_grp_fit_agg).reset_index()

df_w_fit_param_agg = pd.merge(left=temp, right=df_fit_param_agg, how='left', on=cols_to_group_by)
df_w_fit_param_agg.head(3)

,peptide,protein,replicate,product charge,fragment ion,area_ratio,area_min,area_max,peptide_dilution_name,Peptides,...,cov_grad,qtest_grad,mean_intercept,stdv_intercept,cov_intercept,qtest_intercept,mean_r2,stdv_r2,cov_r2,qtest_r2
0,AEFVEVTK,sp|P02769|ALBU_BOVIN,Soroush_SDC_W1730_Sup_D0_column12_2uL_20,1,b2,0.000344,3.157382e+09,1085662.0,AEFVEVTK_D0,AEFVEVTK,...,0.056969,0.0,-0.164569,0.023073,-0.140203,0.0,0.986283,0.002647,0.002684,0.0
1,AEFVEVTK,sp|P02769|ALBU_BOVIN,Soroush_SDC_W1730_Sup_D0_column12_2uL_20,1,y2,0.000128,4.009150e+08,51280.0,AEFVEVTK_D0,AEFVEVTK,...,0.056969,0.0,-0.164569,0.023073,-0.140203,0.0,0.986283,0.002647,0.002684,0.0
2,AEFVEVTK,sp|P02769|ALBU_BOVIN,Soroush_SDC_W1730_Sup_D0_column12_2uL_20,1,y4,0.000017,6.640748e+08,11599.0,AEFVEVTK_D0,AEFVEVTK,...,0.056969,0.0,-0.164569,0.023073,-0.140203,0.0,0.986283,0.002647,0.002684,0.0


In [47]:
# The columns that have zero area as denominator
df_w_fit_param_agg[df_w_fit_param_agg.area_ratio.isna()]

,peptide,protein,replicate,product charge,fragment ion,area_ratio,area_min,area_max,peptide_dilution_name,Peptides,...,cov_grad,qtest_grad,mean_intercept,stdv_intercept,cov_intercept,qtest_intercept,mean_r2,stdv_r2,cov_r2,qtest_r2
855,YICDNQDTISSK,sp|P02769|ALBU_BOVIN,Soroush_SDC_W42_Sup_D2_column12_2uL_99,1,b2,NaN,0.0,104808.0,YICDNQDTISSK_D2,YICDNQDTISSK,...,0.25654,0.0,-4.668425,11.889303,-2.546748,0.529508,NaN,NaN,NaN,0.0
856,YICDNQDTISSK,sp|P02769|ALBU_BOVIN,Soroush_SDC_W42_Sup_D2_column12_2uL_99,1,y10,NaN,0.0,233180.0,YICDNQDTISSK_D2,YICDNQDTISSK,...,0.25654,0.0,-4.668425,11.889303,-2.546748,0.529508,NaN,NaN,NaN,0.0
857,YICDNQDTISSK,sp|P02769|ALBU_BOVIN,Soroush_SDC_W42_Sup_D2_column12_2uL_99,1,y3,NaN,0.0,100127.0,YICDNQDTISSK_D2,YICDNQDTISSK,...,0.25654,0.0,-4.668425,11.889303,-2.546748,0.529508,NaN,NaN,NaN,0.0
858,YICDNQDTISSK,sp|P02769|ALBU_BOVIN,Soroush_SDC_W42_Sup_D2_column12_2uL_99,1,y9,NaN,0.0,42985.0,YICDNQDTISSK_D2,YICDNQDTISSK,...,0.25654,0.0,-4.668425,11.889303,-2.546748,0.529508,NaN,NaN,NaN,0.0
862,YICDNQDTISSK,sp|P02769|ALBU_BOVIN,Soroush_SDC_W42_Sup_D2_column12_2uL_99,2,y10,NaN,0.0,149244.0,YICDNQDTISSK_D2,YICDNQDTISSK,...,0.25654,0.0,-4.668425,11.889303,-2.546748,0.529508,NaN,NaN,NaN,0.0
1025,YICDNQDTISSK,sp|P02769|ALBU_BOVIN,Soroush_SDC_WW_Sup_D2_column12_2uL_101,1,b2,NaN,0.0,212854.0,YICDNQDTISSK_D2,YICDNQDTISSK,...,0.12880,0.0,-7.672875,5.300528,-0.690814,0.394030,NaN,NaN,NaN,0.0
1027,YICDNQDTISSK,sp|P02769|ALBU_BOVIN,Soroush_SDC_WW_Sup_D2_column12_2uL_101,1,y10,NaN,0.0,556007.0,YICDNQDTISSK_D2,YICDNQDTISSK,...,0.12880,0.0,-7.672875,5.300528,-0.690814,0.394030,NaN,NaN,NaN,0.0
1028,YICDNQDTISSK,sp|P02769|ALBU_BOVIN,Soroush_SDC_WW_Sup_D2_column12_2uL_101,1,y3,NaN,0.0,207790.0,YICDNQDTISSK_D2,YICDNQDTISSK,...,0.12880,0.0,-7.672875,5.300528,-0.690814,0.394030,NaN,NaN,NaN,0.0
1029,YICDNQDTISSK,sp|P02769|ALBU_BOVIN,Soroush_SDC_WW_Sup_D2_column12_2uL_101,1,y9,NaN,0.0,127013.0,YICDNQDTISSK_D2,YICDNQDTISSK,...,0.12880,0.0,-7.672875,5.300528,-0.690814,0.394030,NaN,NaN,NaN,0.0
1033,YICDNQDTISSK,sp|P02769|ALBU_BOVIN,Soroush_SDC_WW_Sup_D2_column12_2uL_101,2,y10,NaN,0.0,294669.0,YICDNQDTISSK_D2,YICDNQDTISSK,...,0.12880,0.0,-7.672875,5.300528,-0.690814,0.394030,NaN,NaN,NaN,0.0


In [42]:
result_data_folder = pathlib.Path("../../reports/ms_work")

df_w_fit_param_agg.to_csv(result_data_folder.joinpath('data_with_fit_param_aggregate_soroush_2.csv'), index=False)